In [1]:
!pip install -U conllu datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# Import required libraries
import torch
from torch.utils.data import DataLoader
import transformers
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd
from datasets import load_dataset

In [3]:
german_data = load_dataset("universal_dependencies", "de_gsd")
print(german_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/191k [00:00<?, ?B/s]

universal_dependencies.py:   0%|          | 0.00/87.8k [00:00<?, ?B/s]

The repository for universal_dependencies contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/universal_dependencies.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/13814 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/799 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/977 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 13814
    })
    validation: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 799
    })
    test: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 977
    })
})


In [35]:
class ArcStandardParser:
    def __init__(self):
        self.stack = []
        self.buffer = []
        self.dependencies = []

    def initialize(self, sentence):
        self.stack = [(0, "ROOT")]  # Root represented as index 0
        self.buffer = [(i + 1, token) for i, token in enumerate(sentence)]
        self.dependencies = []

    def shift(self):
        if self.buffer:
            self.stack.append(self.buffer.pop(0))

    def left_arc(self):
        if len(self.stack) >= 2:
            dependent = self.stack.pop(-2)
            head = self.stack[-1]
            self.dependencies.append((head[0], dependent[0]))

    def right_arc(self):
        if len(self.stack) >= 2:
            dependent = self.stack.pop(-1)
            head = self.stack[-1]
            self.dependencies.append((head[0], dependent[0]))

    def parse_step(self, action):
        if action == "SHIFT":
            self.shift()
        elif action == "LEFT-ARC":
            self.left_arc()
        elif action == "RIGHT-ARC":
            self.right_arc()

    def get_state(self):
        return {
            "stack": self.stack,
            "buffer": self.buffer,
            "dependencies": self.dependencies,
        }


In [36]:
def dependency_to_actions(sentence, heads):
    stack = [(0, "ROOT")]
    buffer = []
    for i, token in enumerate(sentence):
        buffer.append((i + 1, token))
    actions = []

    dependents = {}
    for i in range(len(sentence) + 1):  # Include ROOT
        dependents[i] = []

    for i, head in enumerate(heads):
        dependent_token_index = i + 1  # 1-based index for tokens
        dependents[head].append(dependent_token_index)

    created_arcs = set()

    while buffer or len(stack) > 1:
        if len(stack) >= 2:
            s1 = stack[-1][0]
            s2 = stack[-2][0]

            if s1 in dependents[s2]:
                all_dependents_processed = True
                for d in dependents[s1]:
                    if d not in created_arcs:
                        all_dependents_processed = False
                        break
                if all_dependents_processed:
                    actions.append("RIGHT-ARC")
                    created_arcs.add(s1)
                    stack.pop()
                    continue

            if s2 in dependents[s1]:
                all_dependents_processed = True
                for d in dependents[s2]:
                    if d not in created_arcs:
                        all_dependents_processed = False
                        break
                if all_dependents_processed:
                    actions.append("LEFT-ARC")
                    created_arcs.add(s2)
                    stack.pop(-2)
                    continue

        if buffer:
            actions.append("SHIFT")
            next_token = buffer.pop(0)
            stack.append(next_token)
        else:
            break

    return actions

In [38]:
def clean_dataset(sentences, heads):
    """Clean and validate the dataset."""
    cleaned_sentences = []
    cleaned_heads = []
    skipped_indices = []

    for i, (sentence, head) in enumerate(zip(sentences, heads)):
        try:
            head_as_integers = []
            for h in head:
                head_as_integers.append(int(h))
            head = head_as_integers

            head_indices_are_valid = True
            for h in head:
                if h < 0 or h > len(sentence):
                    head_indices_are_valid = False
                    break
            if not head_indices_are_valid:
                skipped_indices.append(i)
                continue

            root_indices = []
            for idx, h in enumerate(head):
                if h == 0:
                    root_indices.append(idx)

            if len(root_indices) != 1:
                skipped_indices.append(i)
                continue

            invalid_head_found = False
            for h in head:
                if h > len(sentence):
                    invalid_head_found = True
                    break
            if invalid_head_found:
                skipped_indices.append(i)
                continue

            cleaned_sentences.append(sentence)
            cleaned_heads.append(head)

        except Exception as e:
            skipped_indices.append(i)

    if len(cleaned_sentences) == 0:
        print("\nExample of first few skipped sentences:")
        for idx in skipped_indices[:3]:
            print(f"\nSentence {idx}:")
            print(f"Text: {sentences[idx]}")
            print(f"Heads: {heads[idx]}")

    return cleaned_sentences, cleaned_heads, skipped_indices

In [39]:
def extract_train_data(dataset):
    train_data = dataset['train']
    sentences = train_data['tokens']
    heads = train_data['head']

    if not isinstance(sentences[0], list):
        sentences = [list(s) for s in sentences]
    if not isinstance(heads[0], list):
        heads = [list(h) for h in heads]
    return sentences, heads

def extract_val_data(dataset):
    val_data = dataset['validation']
    sentences = val_data['tokens']
    heads = val_data['head']
    if not isinstance(sentences[0], list):
        sentences = [list(s) for s in sentences]
    if not isinstance(heads[0], list):
        heads = [list(h) for h in heads]
    return sentences, heads

def extract_test_data(dataset):
    test_data = dataset['test']
    sentences = test_data['tokens']
    heads = test_data['head']
    if not isinstance(sentences[0], list):
        sentences = [list(s) for s in sentences]
    if not isinstance(heads[0], list):
        heads = [list(h) for h in heads]
    return sentences, heads

In [16]:
# Process full datasets
train_sentences, train_heads = extract_train_data(german_data)
clean_train_sent, clean_train_heads, skipped_train = clean_dataset(train_sentences, train_heads)

val_sentences, val_heads = extract_val_data(german_data)
clean_val_sent, clean_val_heads, skipped_val = clean_dataset(val_sentences, val_heads)

test_sentences, test_heads = extract_test_data(german_data)
clean_test_sent, clean_test_heads, skipped_test = clean_dataset(test_sentences, test_heads)

print("\nFinal Dataset Sizes:")
print(f"Train: {len(clean_train_sent)} sentences")
print(f"Validation: {len(clean_val_sent)} sentences")
print(f"Test: {len(clean_test_sent)} sentences")


Final Dataset Sizes:
Train: 10071 sentences
Validation: 657 sentences
Test: 742 sentences


In [17]:
# Test with just first few examples
sample_sentences = train_sentences[:5]
sample_heads = train_heads[:5]

print("Testing cleaning with first 5 sentences:")
clean_test_sent, clean_test_heads, skipped = clean_dataset(sample_sentences, sample_heads)

for i, (sent, heads) in enumerate(zip(clean_test_sent, clean_test_heads)):
    print(f"\nSentence {i}:")
    print(f"Text: {sent}")
    print(f"Heads: {heads}")

Testing cleaning with first 5 sentences:

Sentence 0:
Text: ['Sehr', 'gute', 'Beratung', ',', 'schnelle', 'Behebung', 'der', 'Probleme', ',', 'so', 'stelle', 'ich', 'mir', 'Kundenservice', 'vor', '.']
Heads: [2, 3, 0, 6, 6, 3, 8, 6, 11, 11, 3, 11, 11, 11, 11, 3]

Sentence 1:
Text: ['Nette', 'Gespräche', ',', 'klasse', 'Ergebnis']
Heads: [2, 0, 5, 5, 2]

Sentence 2:
Text: ['Ob', 'bei', 'der', 'Terminvergabe', ',', 'den', 'Behandlungsräumen', 'oder', 'den', 'individuell', 'zugeschnittenen', 'Trainingsplänen', 'sind', 'alle', 'Mitarbeiter', 'äußerst', 'kompetent', 'und', 'flexibel', '.']
Heads: [4, 4, 4, 17, 7, 7, 4, 12, 12, 11, 12, 4, 17, 15, 17, 17, 0, 19, 17, 17]


In [18]:
def generate_training_data(sentences, heads):
    """
    Generate training data by converting all sentences to action sequences.
    """
    training_pairs = []
    for sent, head in zip(sentences, heads):  # head is defined here
        try:
            actions = dependency_to_actions(sent, head)  # should use head not heads
            training_pairs.append((sent, actions))
        except Exception as e:
            print(f"Error: {e}")  # Added error printing for debugging
            continue
    return training_pairs

In [19]:
print("Generating training sequences...")
train_data = generate_training_data(clean_train_sent, clean_train_heads)
val_data = generate_training_data(clean_val_sent, clean_val_heads)
test_data = generate_training_data(clean_test_sent, clean_test_heads)

print(f"Training sequences: {len(train_data)}")
print(f"Validation sequences: {len(val_data)}")
print(f"Test sequences: {len(test_data)}")

# Print example
print("\nExample training sequence:")
example_sent, example_actions = train_data[0]
print(f"Sentence: {example_sent}")
print(f"Action sequence: {example_actions}")

Generating training sequences...
Training sequences: 10071
Validation sequences: 657
Test sequences: 3

Example training sequence:
Sentence: ['Sehr', 'gute', 'Beratung', ',', 'schnelle', 'Behebung', 'der', 'Probleme', ',', 'so', 'stelle', 'ich', 'mir', 'Kundenservice', 'vor', '.']
Action sequence: ['SHIFT', 'SHIFT', 'LEFT-ARC', 'SHIFT', 'LEFT-ARC', 'SHIFT', 'SHIFT', 'SHIFT', 'LEFT-ARC', 'LEFT-ARC', 'SHIFT', 'SHIFT', 'LEFT-ARC', 'RIGHT-ARC', 'RIGHT-ARC', 'SHIFT', 'SHIFT', 'SHIFT', 'LEFT-ARC', 'LEFT-ARC', 'SHIFT', 'RIGHT-ARC', 'SHIFT', 'RIGHT-ARC', 'SHIFT', 'RIGHT-ARC', 'SHIFT', 'RIGHT-ARC', 'RIGHT-ARC', 'SHIFT', 'RIGHT-ARC', 'RIGHT-ARC']


In [20]:
def verify_action_sequence(sentence, heads, actions):
    """
    Verify that an action sequence produces the correct dependencies.
    """
    parser = ArcStandardParser()
    parser.initialize(sentence)

    for action in actions:
        parser.parse_step(action)

    dependencies = parser.get_state()["dependencies"]

    predicted_heads = [0] * len(sentence)
    for head_idx, dependent_idx in dependencies:
        if head_idx == 0:  # ROOT
            predicted_heads[dependent_idx - 1] = 0
        else:
            predicted_heads[dependent_idx - 1] = head_idx

    print("Original heads:", heads)
    print("Predicted heads:", predicted_heads)
    print("Match:", predicted_heads == heads)
    return predicted_heads == heads

# Test
sentence = ['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', '-', 'Ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.']
heads = [0, 1, 1, 1, 6, 7, 1, 7, 8, 8, 8, 8, 8, 15, 8, 18, 18, 7, 21, 21, 18, 23, 21, 21, 28, 28, 28, 21, 1]

# Test with the example
example_sent, example_actions = train_data[0]
example_heads = clean_train_heads[0]

actions = dependency_to_actions(example_sent, example_heads)
verify_action_sequence(example_sent, example_heads, actions)

Original heads: [2, 3, 0, 6, 6, 3, 8, 6, 11, 11, 3, 11, 11, 11, 11, 3]
Predicted heads: [2, 3, 0, 6, 6, 3, 8, 6, 11, 11, 3, 11, 11, 11, 11, 3]
Match: True


True

In [21]:
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [40]:
class DependencyParsingDataset(Dataset):
    def __init__(self, sentences, heads, tokenizer):
        self.sentences = sentences
        self.heads = heads
        self.tokenizer = tokenizer

        self.actions = []
        for s, h in zip(sentences, heads):
            action_sequence = dependency_to_actions(s, h)
            action_sequence_as_int = self.convert_actions_to_int(action_sequence)
            self.actions.append(action_sequence_as_int)

    def convert_actions_to_int(self, actions):
        action_map = {
            "SHIFT": 0,
            "LEFT-ARC": 1,
            "RIGHT-ARC": 2
        }

        actions_as_int = []
        for action in actions:
            actions_as_int.append(action_map[action])

        return actions_as_int

    def __len__(self):
        dataset_length = len(self.sentences)
        return dataset_length

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        actions = self.actions[idx]

        sentence_as_string = " ".join(sentence)
        inputs = self.tokenizer(
            sentence_as_string,
            return_tensors="pt",
            padding=True,
            truncation=True
        )

        token_count = inputs["input_ids"].size(1)

        if len(actions) < token_count:
            padded_actions = actions + [0] * (token_count - len(actions))
            actions = padded_actions
        elif len(actions) > token_count:
            truncated_actions = actions[:token_count]
            actions = truncated_actions

        output = {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "actions": torch.tensor(actions, dtype=torch.long)
        }

        return output


In [41]:
class DependencyParserModel(nn.Module):
    def __init__(self, bert_model_name, num_actions):
        super(DependencyParserModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_actions)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        token_outputs = outputs.last_hidden_state
        action_logits = self.fc(token_outputs)
        return action_logits


In [42]:
def train_model(model, dataloader, optimizer, criterion, num_epochs=3):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            actions = batch["actions"].to(device)

            optimizer.zero_grad()

            logits = model(input_ids, attention_mask)

            batch_size, seq_len, num_actions = logits.size()
            logits = logits.view(-1, num_actions)
            actions = actions.view(-1)

            valid_indices = attention_mask.view(-1).nonzero(as_tuple=True)[0]
            logits = logits[valid_indices]
            actions = actions[valid_indices]

            loss = criterion(logits, actions)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader)}")


In [43]:
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            actions = batch["actions"].to(device)

            logits = model(input_ids, attention_mask)
            predictions = torch.argmax(logits, dim=2)

            valid_indices = attention_mask.view(-1).nonzero(as_tuple=True)[0]
            predictions = predictions.view(-1)[valid_indices]
            actions = actions.view(-1)[valid_indices]

            correct += (predictions == actions).sum().item()
            total += actions.size(0)

    accuracy = correct / total
    print(f"Evaluation Accuracy: {accuracy:.4f}")
    return accuracy


In [44]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
dataset = DependencyParsingDataset(clean_train_sent, clean_train_heads, tokenizer)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
# Model, Optimizer, and Loss Setup
num_actions = 3  # SHIFT, LEFT-ARC, RIGHT-ARC
model = DependencyParserModel("bert-base-uncased", num_actions).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [29]:
for i, batch in enumerate(dataloader):
    print(f"Batch {i + 1}:")
    print("Input IDs shape:", batch["input_ids"].shape)
    print("Attention Mask shape:", batch["attention_mask"].shape)
    print("Actions shape:", batch["actions"].shape)
    print("-" * 40)
    if i == 2:  # Limit to first 3 batches
        break


Batch 1:
Input IDs shape: torch.Size([1, 25])
Attention Mask shape: torch.Size([1, 25])
Actions shape: torch.Size([1, 25])
----------------------------------------
Batch 2:
Input IDs shape: torch.Size([1, 45])
Attention Mask shape: torch.Size([1, 45])
Actions shape: torch.Size([1, 45])
----------------------------------------
Batch 3:
Input IDs shape: torch.Size([1, 52])
Attention Mask shape: torch.Size([1, 52])
Actions shape: torch.Size([1, 52])
----------------------------------------


In [30]:
# Iterate over the DataLoader and print a few batches
for i, batch in enumerate(dataloader):
    print(f"Batch {i + 1}:")
    print("Input IDs:", batch["input_ids"])
    print("Attention Mask:", batch["attention_mask"])
    print("Actions:", batch["actions"])
    print("-" * 40)  # Separator for readability
    if i == 2:  # Limit to the first 3 batches
        break


Batch 1:
Input IDs: tensor([[  101,  1041, 10609, 28067,  3150, 27829,  2053,  2818, 27665,  2061,
          6914, 11639,  2618,  1000, 19204,  1011,  3229,  1011,  4118,  2063,
          1000,  7632, 14191,  2226,  1012,   102]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])
Actions: tensor([[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 2, 0, 2,
         2, 0]])
----------------------------------------
Batch 2:
Input IDs: tensor([[  101,  3614,  5484,  2638, 12170, 28476, 27665,  2078, 16216, 11624,
         10483,  5054,  2368, 16853, 24488,  1047,  2890,  2483,  6151, 11417,
          5054,  4830,  5886, 27617, 16216,  4892,  2063, 16853, 13719,  2358,
          2890, 16093, 14273,  2121, 21200,  1012,   102]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Actions: tensor([[0, 0, 1, 0, 0, 0, 0, 1, 1

In [33]:
train_model(model, dataloader, optimizer, criterion, num_epochs=3)

Epoch 1/3, Loss: 0.5180137335541641
Epoch 2/3, Loss: 0.4634156874138257
Epoch 3/3, Loss: 0.41863586920577706


In [34]:
evaluate_model(model, dataloader)

Evaluation Accuracy: 0.8069


0.806945865826795